# 01_TGU_validation.ipynb

**Validação do modelo TGU para precessão orbital**

Notebook para reproduzir as predições do artigo:
"A Unified Informational Model for Orbital Precession: Spin-Informational Corrections from the TGU Framework"
(Henry Matuchaki, 2025)

Objetivos:
- Carregar dados orbitais
- Calcular fator α = 1 + k · (e/a)
- Aplicar correção na precessão GR
- Comparar com valores do paper
- Visualizar α em função da tensão informacional (e/a)

In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Importar funções do pacote TGU
from src.tgu_precession import tgu_alpha, tgu_precession, DEFAULT_K

print(f"Matuchaki Parameter k = {DEFAULT_K:.6f}")

## 1. Carregar dados orbitais

In [ ]:
df_orbital = pd.read_csv('../data/orbital_data.csv')
df_orbital

## 2. Calcular predições TGU para todos os corpos

In [ ]:
results = []

for _, row in df_orbital.iterrows():
    body = row['body']
    e = row['eccentricity_e']
    a = row['semi_major_axis_a_AU']
    delta_gr = row['delta_phi_GR_arcsec_century']
    
    alpha = tgu_alpha(e, a)
    delta_tgu = tgu_precession(e, a, delta_gr)
    strain = e / a
    
    results.append({
        'Body': body,
        'e': e,
        'a (AU)': a,
        'Strain (e/a)': round(strain, 4),
        'α': round(alpha, 5),
        'Δϕ_GR (arcsec/cent)': delta_gr,
        'Δϕ_TGU (arcsec/cent)': round(delta_tgu, 2)
    })

df_results = pd.DataFrame(results)
df_results

## 3. Comparação visual: α em função da tensão informacional (e/a)

In [ ]:
plt.figure(figsize=(10, 6))

plt.scatter(df_results['Strain (e/a)'], df_results['α'], 
            s=100, c='darkblue', edgecolor='white', linewidth=1.5)

for i, row in df_results.iterrows():
    plt.annotate(row['Body'], (row['Strain (e/a)'], row['α']), 
                 xytext=(5, 5), textcoords='offset points', fontsize=10)

plt.axhline(1.0, color='gray', linestyle='--', alpha=0.7, label='Sem correção (GR puro)')

plt.xlabel('Tensão Informacional (e/a)', fontsize=12)
plt.ylabel('Fator de Correção α', fontsize=12)
plt.title('Correção TGU α vs. Tensão Orbital (k = 0.0881)', fontsize=14)
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

## 4. Observações finais

- Para órbitas quase circulares (baixa strain): α ≈ 1 → convergência com GR
- Para alta excentricidade (ex.: HD 80606b, Icarus): correção significativa (α > 1.05)
- Próximos passos: integrar simulações de spin networks (02_k_derivation_spin.ipynb)
  e testar com mais exoplanetas ou pulsares binários